In [5]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import ParameterSampler
from tqdm import tqdm
import mlflow
import mlflow.prophet

# Load data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

# Store predictions
all_forecasts = []

# Define hyperparameter space
param_grid = {
    'changepoint_prior_scale': [0.01, 0.05, 0.1],
    'weekly_seasonality': [True, False],
    'yearly_seasonality': [True, False],
}

# Sample combinations
param_samples = list(ParameterSampler(param_grid, n_iter=12, random_state=42))

# Unique (store, product) pairs
pairs = train[['store', 'product']].drop_duplicates()

# Enable MLflow experiment
mlflow.set_experiment("Sales Forecasting Models")

# Loop through each pair
for _, row in tqdm(pairs.iterrows(), total=len(pairs)):
    store_id = row['store']
    product_id = row['product']

    with mlflow.start_run(run_name=f"Prophet_store_{store_id}_product_{product_id}"):
        # Filter data for current pair
        train_pair = train[(train['store'] == store_id) & (train['product'] == product_id)].copy()
        test_pair = test[(test['store'] == store_id) & (test['product'] == product_id)].copy()

        # Rename for Prophet
        train_pair = train_pair.rename(columns={'Date': 'ds', 'number_sold': 'y'})

        # Train/validation split
        validation_start = pd.to_datetime('2018-01-01')
        train_data = train_pair[train_pair['ds'] < validation_start]
        val_data = train_pair[train_pair['ds'] >= validation_start]

        best_rmse = float('inf')
        best_model = None
        best_params = None

        # Try each parameter set
        for params in param_samples:
            model = Prophet(
                changepoint_prior_scale=params['changepoint_prior_scale'],
                weekly_seasonality=params['weekly_seasonality'],
                yearly_seasonality=params['yearly_seasonality'],
            )

            try:
                model.fit(train_data)

                # Predict validation period
                future_val = val_data[['ds']]
                forecast_val = model.predict(future_val)
                y_pred = forecast_val['yhat'].values
                y_true = val_data['y'].values

                rmse = mean_squared_error(y_true, y_pred, squared=False)

                if rmse < best_rmse:
                    best_rmse = rmse
                    mae = mean_absolute_error(y_true, y_pred)
                    best_params = params

            except Exception as e:
                continue

        if best_params is None:
            mlflow.log_param("status", "no_valid_model")
            continue

        # Log best hyperparameters and metric
        mlflow.log_params({
            "store": store_id,
            "product": product_id,
            **best_params
        })
        mlflow.log_metric("val_rmse", best_rmse)
        mlflow.log_metric("val_mae", mae)

        # Train best model on full training set
        best_model = Prophet(
            changepoint_prior_scale=best_params['changepoint_prior_scale'],
            weekly_seasonality=best_params['weekly_seasonality'],
            yearly_seasonality=best_params['yearly_seasonality'],
        )
        best_model.fit(train_pair)

        # Log model
        mlflow.prophet.log_model(best_model, artifact_path="model")
        mlflow.set_tag("model", "Prophet")
        mlflow.set_tag("store_id", store_id)
        mlflow.set_tag("product_id", product_id)

        # Forecast on test period (2019)
        future_test = test_pair[['Date']].rename(columns={'Date': 'ds'}).drop_duplicates()
        forecast_test = best_model.predict(future_test)
        forecast_test = forecast_test[['ds', 'yhat']].rename(columns={'ds': 'Date', 'yhat': 'predicted_number_sold'})

        forecast_test['store'] = store_id
        forecast_test['product'] = product_id

        # Join with original test to preserve structure
        merged = test_pair.merge(forecast_test, on=['Date', 'store', 'product'], how='left')
        all_forecasts.append(merged)
        
        rmse_test = mean_squared_error(merged['number_sold'], merged['number_sold'], squared=False)
        mae_test = mean_absolute_error(merged['number_sold'], merged['number_sold'])
        
        mlflow.log_metric("test_rmse", rmse_test)
        mlflow.log_metric("test_mae", mae_test)
        
        
# Combine all
final_result = pd.concat(all_forecasts)
final_result = final_result[['Date', 'store', 'product', 'number_sold', 'predicted_number_sold']]

# Log final RMSE across all pairs
final_rmse = mean_squared_error(final_result['number_sold'], final_result['predicted_number_sold'], squared=False)
final_mae = mean_absolute_error(final_result['number_sold'], final_result['predicted_number_sold'])


mlflow.set_experiment("Final Aggregated Forecasting")
with mlflow.start_run(run_name="Overall RMSE Summary Test Prophet"):
    mlflow.log_metric("final_rmse", final_rmse)
    
    final_result.to_csv("final_predictions_prophet.csv", index=False)
    mlflow.log_artifact("final_predictions_prophet.csv")

print("Final RMSE:", final_rmse)
print("Final MAE:", final_mae)

  0%|                                                                                           | 0/70 [00:00<?, ?it/s]11:48:30 - cmdstanpy - INFO - Chain [1] start processing
11:48:30 - cmdstanpy - INFO - Chain [1] done processing
11:48:30 - cmdstanpy - INFO - Chain [1] start processing
11:48:30 - cmdstanpy - INFO - Chain [1] done processing
11:48:31 - cmdstanpy - INFO - Chain [1] start processing
11:48:31 - cmdstanpy - INFO - Chain [1] done processing
11:48:31 - cmdstanpy - INFO - Chain [1] start processing
11:48:31 - cmdstanpy - INFO - Chain [1] done processing
11:48:32 - cmdstanpy - INFO - Chain [1] start processing
11:48:32 - cmdstanpy - INFO - Chain [1] done processing
11:48:32 - cmdstanpy - INFO - Chain [1] start processing
11:48:33 - cmdstanpy - INFO - Chain [1] done processing
11:48:33 - cmdstanpy - INFO - Chain [1] start processing
11:48:34 - cmdstanpy - INFO - Chain [1] done processing
11:48:34 - cmdstanpy - INFO - Chain [1] start processing
11:48:35 - cmdstanpy - INFO - Cha

11:49:24 - cmdstanpy - INFO - Chain [1] start processing
11:49:25 - cmdstanpy - INFO - Chain [1] done processing
11:49:25 - cmdstanpy - INFO - Chain [1] start processing
11:49:26 - cmdstanpy - INFO - Chain [1] done processing
11:49:26 - cmdstanpy - INFO - Chain [1] start processing
11:49:27 - cmdstanpy - INFO - Chain [1] done processing
11:49:27 - cmdstanpy - INFO - Chain [1] start processing
11:49:27 - cmdstanpy - INFO - Chain [1] done processing
11:49:28 - cmdstanpy - INFO - Chain [1] start processing
11:49:28 - cmdstanpy - INFO - Chain [1] done processing
11:49:29 - cmdstanpy - INFO - Chain [1] start processing
11:49:29 - cmdstanpy - INFO - Chain [1] done processing
11:49:30 - cmdstanpy - INFO - Chain [1] start processing
11:49:30 - cmdstanpy - INFO - Chain [1] done processing
11:49:31 - cmdstanpy - INFO - Chain [1] start processing
11:49:31 - cmdstanpy - INFO - Chain [1] done processing
2025/07/11 11:49:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name

11:50:11 - cmdstanpy - INFO - Chain [1] start processing
11:50:11 - cmdstanpy - INFO - Chain [1] done processing
11:50:12 - cmdstanpy - INFO - Chain [1] start processing
11:50:12 - cmdstanpy - INFO - Chain [1] done processing
2025/07/11 11:50:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/11 11:50:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
 13%|██████████▋                                                                        | 9/70 [01:43<11:01, 10.85s/it]11:50:13 - cmdstanpy - INFO - Chain [1] start processing
11:50:14 - cmdstanpy - INFO - Chain [1] done processing
11:50:14 - cmdstanpy - INFO - Chain [1] start processing
11:50:14 - cmdstanpy - INFO - Chain [1] done processing
11:50:15 - cmdstanpy - INFO - Chain [1] start processing
11:50:15 - cmdstanpy - INFO - Chain [1] done processing
11:50:15 - cm

 19%|███████████████▏                                                                  | 13/70 [02:27<10:19, 10.87s/it]11:50:57 - cmdstanpy - INFO - Chain [1] start processing
11:50:57 - cmdstanpy - INFO - Chain [1] done processing
11:50:58 - cmdstanpy - INFO - Chain [1] start processing
11:50:58 - cmdstanpy - INFO - Chain [1] done processing
11:50:58 - cmdstanpy - INFO - Chain [1] start processing
11:50:58 - cmdstanpy - INFO - Chain [1] done processing
11:50:59 - cmdstanpy - INFO - Chain [1] start processing
11:50:59 - cmdstanpy - INFO - Chain [1] done processing
11:50:59 - cmdstanpy - INFO - Chain [1] start processing
11:51:00 - cmdstanpy - INFO - Chain [1] done processing
11:51:00 - cmdstanpy - INFO - Chain [1] start processing
11:51:01 - cmdstanpy - INFO - Chain [1] done processing
11:51:01 - cmdstanpy - INFO - Chain [1] start processing
11:51:02 - cmdstanpy - INFO - Chain [1] done processing
11:51:02 - cmdstanpy - INFO - Chain [1] start processing
11:51:03 - cmdstanpy - INFO - Cha

11:51:42 - cmdstanpy - INFO - Chain [1] start processing
11:51:43 - cmdstanpy - INFO - Chain [1] done processing
11:51:43 - cmdstanpy - INFO - Chain [1] start processing
11:51:44 - cmdstanpy - INFO - Chain [1] done processing
11:51:44 - cmdstanpy - INFO - Chain [1] start processing
11:51:45 - cmdstanpy - INFO - Chain [1] done processing
11:51:45 - cmdstanpy - INFO - Chain [1] start processing
11:51:45 - cmdstanpy - INFO - Chain [1] done processing
11:51:46 - cmdstanpy - INFO - Chain [1] start processing
11:51:46 - cmdstanpy - INFO - Chain [1] done processing
11:51:46 - cmdstanpy - INFO - Chain [1] start processing
11:51:47 - cmdstanpy - INFO - Chain [1] done processing
11:51:47 - cmdstanpy - INFO - Chain [1] start processing
11:51:47 - cmdstanpy - INFO - Chain [1] done processing
11:51:48 - cmdstanpy - INFO - Chain [1] start processing
11:51:48 - cmdstanpy - INFO - Chain [1] done processing
2025/07/11 11:51:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name

...

11:58:02 - cmdstanpy - INFO - Chain [1] start processing
11:58:02 - cmdstanpy - INFO - Chain [1] done processing
11:58:03 - cmdstanpy - INFO - Chain [1] start processing
11:58:03 - cmdstanpy - INFO - Chain [1] done processing
11:58:03 - cmdstanpy - INFO - Chain [1] start processing
11:58:04 - cmdstanpy - INFO - Chain [1] done processing
11:58:04 - cmdstanpy - INFO - Chain [1] start processing
11:58:05 - cmdstanpy - INFO - Chain [1] done processing
11:58:05 - cmdstanpy - INFO - Chain [1] start processing
11:58:06 - cmdstanpy - INFO - Chain [1] done processing
11:58:06 - cmdstanpy - INFO - Chain [1] start processing
11:58:07 - cmdstanpy - INFO - Chain [1] done processing
11:58:07 - cmdstanpy - INFO - Chain [1] start processing
11:58:08 - cmdstanpy - INFO - Chain [1] done processing
11:58:09 - cmdstanpy - INFO - Chain [1] start processing
11:58:09 - cmdstanpy - INFO - Chain [1] done processing
11:58:10 - cmdstanpy - INFO - Chain [1] start processing
11:58:11 - cmdstanpy - INFO - Chain [1]

11:59:02 - cmdstanpy - INFO - Chain [1] start processing
11:59:02 - cmdstanpy - INFO - Chain [1] done processing
11:59:03 - cmdstanpy - INFO - Chain [1] start processing
11:59:03 - cmdstanpy - INFO - Chain [1] done processing
11:59:04 - cmdstanpy - INFO - Chain [1] start processing
11:59:04 - cmdstanpy - INFO - Chain [1] done processing
11:59:05 - cmdstanpy - INFO - Chain [1] start processing
11:59:05 - cmdstanpy - INFO - Chain [1] done processing
11:59:05 - cmdstanpy - INFO - Chain [1] start processing
11:59:06 - cmdstanpy - INFO - Chain [1] done processing
11:59:06 - cmdstanpy - INFO - Chain [1] start processing
11:59:06 - cmdstanpy - INFO - Chain [1] done processing
2025/07/11 11:59:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/11 11:59:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
 87%|██████████████

2025/07/11 11:59:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/11 11:59:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
 93%|████████████████████████████████████████████████████████████████████████████▏     | 65/70 [11:25<00:59, 11.81s/it]11:59:55 - cmdstanpy - INFO - Chain [1] start processing
11:59:55 - cmdstanpy - INFO - Chain [1] done processing
11:59:55 - cmdstanpy - INFO - Chain [1] start processing
11:59:56 - cmdstanpy - INFO - Chain [1] done processing
11:59:56 - cmdstanpy - INFO - Chain [1] start processing
11:59:56 - cmdstanpy - INFO - Chain [1] done processing
11:59:57 - cmdstanpy - INFO - Chain [1] start processing
11:59:57 - cmdstanpy - INFO - Chain [1] done processing
11:59:57 - cmdstanpy - INFO - Chain [1] start processing
11:59:58 - cmdstanpy - INFO - Chain [1] done processing
11:59:58 - cm

12:00:42 - cmdstanpy - INFO - Chain [1] start processing
12:00:42 - cmdstanpy - INFO - Chain [1] done processing
12:00:43 - cmdstanpy - INFO - Chain [1] start processing
12:00:43 - cmdstanpy - INFO - Chain [1] done processing
12:00:43 - cmdstanpy - INFO - Chain [1] start processing
12:00:44 - cmdstanpy - INFO - Chain [1] done processing
12:00:44 - cmdstanpy - INFO - Chain [1] start processing
12:00:45 - cmdstanpy - INFO - Chain [1] done processing
12:00:45 - cmdstanpy - INFO - Chain [1] start processing
12:00:45 - cmdstanpy - INFO - Chain [1] done processing
12:00:46 - cmdstanpy - INFO - Chain [1] start processing
12:00:46 - cmdstanpy - INFO - Chain [1] done processing
12:00:47 - cmdstanpy - INFO - Chain [1] start processing
12:00:47 - cmdstanpy - INFO - Chain [1] done processing
12:00:48 - cmdstanpy - INFO - Chain [1] start processing
12:00:48 - cmdstanpy - INFO - Chain [1] done processing
12:00:49 - cmdstanpy - INFO - Chain [1] start processing
12:00:49 - cmdstanpy - INFO - Chain [1]

Final RMSE: 15.817414174944203
Final MAE: 11.848255717363768


In [6]:
final_result

,Date,store,product,number_sold,predicted_number_sold
0,2019-01-01,0,0,845,843.041792
1,2019-01-02,0,0,851,843.405028
2,2019-01-03,0,0,840,843.936804
3,2019-01-04,0,0,842,844.387879
4,2019-01-05,0,0,845,845.303260
...,...,...,...,...,...
360,2019-12-27,6,9,901,908.289839
361,2019-12-28,6,9,926,914.560141
362,2019-12-29,6,9,913,911.484793
363,2019-12-30,6,9,899,900.870757


In [7]:
mean_squared_error(final_result['number_sold'], final_result['predicted_number_sold'], squared=False)

15.817414174944203